## 기본코드
> 무조건 실행


In [ ]:
!pip install geemap
#!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
!pip install pandas
!pip install geopandas
!pip install mapclassify
!pip install numpy==1.23.5
!pip install tqdm
!pip install mss
!pip install pgeocode pandas geopandas torch pytorch-forecasting statsmodels optuna-integration
!pip install PublicDataReader

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 108.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/new_crop"

BASE_PATH = os.path.abspath(BASE_PATH)

DATA_PATH = os.path.join(BASE_PATH, "dataset")

STAELITE_PATH = os.path.join(DATA_PATH, 'SATELITE')


In [ ]:
#%load_ext cudf.pandas
import pandas as pd
import numpy as np
import geopandas as gpd
import tqdm, os, ee, geemap, datetime, joblib, pgeocode, glob, os
import PublicDataReader as pdr

# Raw Dataset
#raw_df = pd.read_csv(os.path.join(DATA_PATH, 'annualy.csv'))

# GEOID -> Location
#gdf = gpd.read_file(os.path.join(DATA_PATH,"tl_2024_us_county.shp"))

### 설정

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-kuuwange')

YEAR = 2016

행정동코드 = pdr.code_hdong()
행정동코드.head()

법정동코드 = pdr.code_bdong()
법정동코드.head()

연결정보 = pdr.code_hdong_bdong()
연결정보.head()

연결정보 = 연결정보#[['행정동코드', '법정동코드', '읍면동명']]

연결정보.sample(10)

#### GEE 관련 함수

In [ ]:
# NDVI / NDRE / EVI 지수 추가

def add_ndvi(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def add_ndre(image):
  ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE');
  return image.addBands(ndre)

def add_evi(image):
  evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
      'NIR': image.select('B8'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
    }).rename('EVI');
  return image.addBands(evi)

images = None
modis_images = None

## 이미지 년도에 맞춰 다시 가져오도록
def update_image(year):
  global images, cleaned_collection, ndvi_median, ndre_median, evi_median, modis_images
  start_date = f"{year:04d}-04-01"
  end_date = f"{year:04d}-08-31"
  images = (
      ee.ImageCollection('NASA/HLS/HLSS30/v002') \
      .filterBounds(ee.Geometry.Rectangle([125.991211,33.063924, 129.440918,38.754083])) \
      .filterDate(start_date, end_date) \
      .filter(ee.Filter.lt("CLOUD_COVERAGE", 10)) \
  )

  modis_images = (
      ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
      .filterDate(start_date, end_date) \
      .filterBounds(ee.Geometry.Rectangle([125.991211,33.063924, 129.440918,38.754083]))
      # .filter(ee.Filter.lt("CLOUD_COVERAGE", 10))
  )

  cleaned_collection = images \
      .map(add_ndvi) \
      .map(add_ndre) \
      .map(add_evi)
  ndvi_median = cleaned_collection.select('NDVI').median()
  ndre_median = cleaned_collection.select('NDRE').median()
  evi_median = cleaned_collection.select('EVI').median()



update_image(2016)

## GPD 생성
>
> 행정동 데이터 -> 행정동 리스트 -> 법정동 코드 변환 및 결합

#### 생성코드
> 데이터 생성이 필요할경우

##### BOUNDARY_GDF 변환

In [ ]:
# BOUNDARY_GDF의 내용을 변환하고 재정리
boundary_gdf = boundary_gdf[boundary_gdf['farmmap'].apply(lambda x: len(x) > 0)]
def extract_farmmap_info(farmmap_list):
    return [
        {
            'UID': item.get('UID'),
            'geometry': item.get('geometry'),
            '면적': item.get('면적'),
            '법정동주소': item.get('법정동주소'),
            '팜맵ID': item.get('팜맵ID')
        }
        for item in farmmap_list
        if isinstance(item, dict)
    ]

boundary_gdf['farmmap'] = boundary_gdf['farmmap'].apply(extract_farmmap_info)

NameError: name 'boundary_gdf' is not defined

In [ ]:
collapsed_gdf = gpd.read_feather(os.path.join(DATA_PATH, 'SATELITE', f'NDVI_{YEAR}.feather'))

#collapsed_gdf = collapsed_gdf[['adm_cd', 'value', 'name']]



In [ ]:

farmmap_collapsed = collapsed_gdf.merge(boundary_gdf, on='adm_cd', how='right')
farmmap_collapsed.to_pickle(os.path.join(DATA_PATH, 'farmmap_collapsed.pkl'), protocol=5, compression='gzip')

print (farmmap_collapsed.size)
farmmap_collapsed.sample(2)

24505


,adm_cd,name,value,geometry_x,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,geometry_y,bjdcode,farmmap
674,33030730,충청북도 제천시 의림지동,0.59979,"MULTIPOLYGON (((128.24467 37.15388, 128.24458 ...",충청북도 제천시 의림지동,4315051800,43150,43,충청북도,제천시,"MULTIPOLYGON (((128.24467 37.15388, 128.24458 ...","[4315011400, 4315011500]","[{'UID': '11940913', 'geometry': [{'type': 'Mu..."
1319,37040520,경상북도 안동시 명륜동,0.36518,"MULTIPOLYGON (((128.75838 36.58401, 128.75887 ...",경상북도 안동시 명륜동,4717052000,47170,47,경상북도,안동시,"MULTIPOLYGON (((128.75838 36.58401, 128.75887 ...","[4717010400, 4717010500, 4717013100, 4717013200]","[{'UID': '04606786', 'geometry': [{'type': 'Mu..."


##### MULTIPOLYGON UTIL

In [ ]:
import geemap
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
import json

def convert_to_multipolygon(geom):
  multipolygon_coords = []
  for polygon in geom:
    coords = [(pt['x'], pt['y']) for pt in polygon['xy']]
    multipolygon_coords.append([coords])  # Single polygon ring
  return MultiPolygon([Polygon(p[0]) for p in multipolygon_coords])

def convert_to_gps_polygon(raw_data):
  # shapely MultiPolygon으로 변환
  # GeoDataFrame 생성
  features = []
  for index, row in raw_data.iterrows():
    print (f"\r{index}/{raw_data.shape[0]}", end="")
    #target_row = row['farmmap']
    # print (item)
    target_row = row['farmmap']
    print (target_row)
    for item in target_row:
      geom = convert_to_multipolygon(item['geometry'])
      props = {k: v for k, v in item.items() if k != 'geometry'}
      props['geometry'] = geom
      features.append(props)
  farmmap_gdf = gpd.GeoDataFrame(features, crs='EPSG:5179')  # TM 좌표계 (Korea) / 팜맵 전역에서 5179 좌표계 사용중

  farmmap_gdf_4326 = farmmap_gdf.to_crs(epsg=4326) # 위경도 재투영
  return farmmap_gdf_4326


#### 저장

In [ ]:
#farmmap_collapsed = pd.read_pickle(os.path.join(DATA_PATH, 'farmmap_collapsed.pkl'))
farmmap_raw = farmmap_collapsed#.head(500)
farmmap_gdf = convert_to_gps_polygon(farmmap_raw)
FARMMAP_JSON_PATH = os.path.join(DATA_PATH, "farmmap.geojson")
farmmap_gdf.to_file(FARMMAP_JSON_PATH, driver="GeoJSON")

In [ ]:
farmmap_collapsed.sample(30).to_dict(orient='records')

In [ ]:
#farmmap_gdf = gpd.read_file(FARMMAP_JSON_PATH)
def get_related_farmmap(x):
  uid_list = [y['UID'] for y in x]
  #print (uid_list)
  matched_farmmap_gdf = farmmap_gdf[farmmap_gdf['UID'].isin(uid_list)]
  x = farmmap_gdf.to_json(ensure_ascii=False,indent=None)
  return str(x)


farmmap_collapsed['new_fm'] = farmmap_collapsed['farmmap'].apply(get_related_farmmap)
#farmmap_collapsed.dtypes['new_fm'] = farmmap_collapsed['new_fm'].astype(str)
#print(farmmap_collapsed['farmmap'].iloc[0:15].apply(get_related_farmmap).iloc[10])

In [ ]:
#farmmap_collapsed['farmmap'].iloc[0:15].apply(get_related_farmmap).iloc[10]
#farmmap_collapsed.sample(1)
# 중복된 geometry중 하나 제거
#farmmap_collapsed = farmmap_collapsed

farmmap_collapsed.sample(2)['new_fm']

In [ ]:
#farmmap_collapsed.to_csv(os.path.join(DATA_PATH, 'all_farmmap_collapsed.csv'))

#x = gpd.read_file(os.path.join(DATA_PATH, 'all_farmmap_collapsed.csv'))
feather_file_path = os.path.join(DATA_PATH, '행정동_법정동결합_farmmap_cleaned.feather')

boundary_gdf = gpd.read_feather(feather_file_path)
boundary_gdf = boundary_gdf[boundary_gdf['bjdcode'].apply(lambda x: len(x) > 0)]

x = boundary_gdf.copy()
x = x.rename(columns={'field_1': 'filedId', 'geometry_x': 'geometry'})
x = x[['adm_cd', 'name', 'geometry', 'adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'bjdcode', 'new_fm']]
x.to_pickle(os.path.join(DATA_PATH, 'all_farmmap_collapsed.pkl'), protocol=5, compression='gzip')
x.to_csv(os.path.join(DATA_PATH, 'all_farmmap_collapsed.csv'))
x.to_feather(os.path.join(DATA_PATH, 'all_farmmap_collapsed.feather'))

KeyError: "['new_fm'] not in index"

#### 사용

In [ ]:
FARMMAP_JSON_PATH = os.path.join(DATA_PATH, "farmmap.geojson")
#farmmap_gdf.to_file(FARMMAP_JSON_PATH, driver="GeoJSON")
farmmap_gdf = gpd.read_file(FARMMAP_JSON_PATH)

#feather_file_path = os.path.join(DATA_PATH, '행정동_법정동결합_farmmap.feather')
feather_file_path = os.path.join(DATA_PATH, '행정동_법정동결합_farmmap_cleaned.feather')

boundary_gdf = gpd.read_feather(feather_file_path)
#boundary_gdf = boundary_gdf[boundary_gdf['bjdcode'].apply(lambda x: len(x) > 0)]

boundary_gdf.sample(3)

gdf = gpd.read_file(os.path.join(DATA_PATH, 'all_farmmap_collapsed.csv'))

In [ ]:
gdf.sample(1)

##### 사용코드
> 생성없이 사용

In [ ]:
### 행정동별 논/밭

FARMMAP_JSON_PATH = os.path.join(DATA_PATH, "farmmap.geojson")
#farmmap_gdf.to_file(FARMMAP_JSON_PATH, driver="GeoJSON")
farmmap_gdf = gpd.read_file(FARMMAP_JSON_PATH)

#feather_file_path = os.path.join(DATA_PATH, '행정동_법정동결합_farmmap.feather')
feather_file_path = os.path.join(DATA_PATH, '행정동_법정동결합_farmmap_cleaned.feather')

boundary_gdf = gpd.read_feather(feather_file_path)
#boundary_gdf = boundary_gdf[boundary_gdf['bjdcode'].apply(lambda x: len(x) > 0)]

boundary_gdf.sample(3)

## NDVI + FARMMAP + GPS 시각화

### VISUALIZATION

In [ ]:
farmmap_gdf.sample(1)

##### 행정동 + 농장별


In [ ]:
visualization = {
    'min': 0.01,
    'max': 0.18,
    'bands': ['B4', 'B3', 'B2'],
}



print(f"NA값 : {collapsed_gdf.isna().sum()}")
choropleth_gdf = collapsed_gdf.dropna()
choropleth_gdf.reset_index(drop=True, inplace=True)

m_ndvi = geemap.Map(width="1000px", height="1200px")

s2_image = images.median()

m_ndvi.add_layer(s2_image, visualization, 'RGB')
m_ndvi.add_layer(ndvi_median, {'min': 0.0, 'max': 0.9, 'palette': ['brown', 'yellow', 'green']}, 'NDVI')
#m_ndvi.add_layer(ndre_median, {'min': 0.0, 'max': 0.5, 'palette': ['purple', 'white', 'darkgreen']}, 'NDRE')
#m_ndvi.add_layer(evi_median, {'min': 0.0, 'max': 1.0, 'palette': ['black', 'lightblue', 'lime']}, 'EVI')

# m_ndvi.add_data(
#     choropleth_gdf,
#     column='value',
#     layer_name='행정동 NDVI',
#     cmap='BrBG',
#     legend_title='NDVI (평균)',
#     style={'color': 'black', 'weight': 0.5, 'fillOpacity': 0.7}
# )

ndvi_vis = {
    'min': 0.0,
    'max': 0.9,
    'palette': ['brown', 'yellow', 'green']
}

#m_ndvi.add_time_slider(cleaned_collection.select('NDVI'), vis_params=ndvi_vis, time_interval=2, date_format='YYYY-MM-dd', layer_name='NDVI 시계열')

#m_ndvi.add_geojson(FARMMAP_JSON_PATH, layer_name="농지경계", style={'color': 'yellow', 'fillColor': 'yellow', 'fillOpacity': 0.5})

m = m_ndvi

m

##### 전체 합본

In [ ]:
visualization = {
    'min': 0.01,
    'max': 0.18,
    'bands': ['B4', 'B3', 'B2'],
}



print(f"NA값 : {collapsed_gdf.isna().sum()}")
choropleth_gdf = collapsed_gdf.dropna()
choropleth_gdf.reset_index(drop=True, inplace=True)

m_ndvi = geemap.Map(width="1000px", height="1200px")

s2_image = images.median()

m_ndvi.add_layer(s2_image, visualization, 'RGB')
m_ndvi.add_layer(ndvi_median, {'min': 0.0, 'max': 0.9, 'palette': ['brown', 'yellow', 'green']}, 'NDVI')
#m_ndvi.add_layer(ndre_median, {'min': 0.0, 'max': 0.5, 'palette': ['purple', 'white', 'darkgreen']}, 'NDRE')
#m_ndvi.add_layer(evi_median, {'min': 0.0, 'max': 1.0, 'palette': ['black', 'lightblue', 'lime']}, 'EVI')

# m_ndvi.add_data(
#     choropleth_gdf,
#     column='value',
#     layer_name='행정동 NDVI',
#     cmap='BrBG',
#     legend_title='NDVI (평균)',
#     style={'color': 'black', 'weight': 0.5, 'fillOpacity': 0.7}
# )

ndvi_vis = {
    'min': 0.0,
    'max': 0.9,
    'palette': ['brown', 'yellow', 'green']
}

m_ndvi.add_time_slider(cleaned_collection.select('NDVI'), vis_params=ndvi_vis, time_interval=2, date_format='YYYY-MM-dd', layer_name='NDVI 시계열')

#m_ndvi.add_gdf(farmmap_gdf, layer_name="농지", style={'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.5})

import ipywidgets as widgets
from IPython.display import display


capture_btn = widgets.Button(description='지도 캡처', icon='camera', button_style='success')
CAPTURE_COUNT_NDVI = 0

# 캡처 동작 연결
def capture_map(btn):
  global CAPTURE_COUNT_NDVI
  filename = f"CAPTURE_{CAPTURE_COUNT_NDVI}.png"
  filename = os.path.join(DATA_PATH, 'SATELITE', 'NDVI', filename)

  m_ndvi.to_image(filename=filename, display=False)
  print(f"✅ 지도 캡처 저장됨: {filename}")

capture_btn.on_click(capture_map)

# 버튼과 지도 함께 표시
control_panel = widgets.HBox([capture_btn])

display(control_panel)
display(m_ndvi)

In [ ]:
m_ndvi

## 학습 및 추론

In [ ]:
import geemap, os
import geopandas as gpd

json_file_path = os.path.join(DATA_PATH, '행정동경계.geojson')
gdf = gpd.read_file(json_file_path)

product_df = pd.read_csv(os.path.join(DATA_PATH, '논벼_파종수확_위경도.csv'), index_col=0)
product_df = product_df[['시도', '시군구', '연도', '면적_ha', '위도', '경도', '10a당_생산량_kg']]

product_df.sort_values(by=['면적_ha'])

,시도,시군구,연도,면적_ha,위도,경도,10a당_생산량_kg
162,경기도,과천시,2016,1.0,37.429233,126.987721,497.0
1345,경기도,과천시,2024,3.0,37.429233,126.987721,470.0
1337,경기도,성남시,2024,3.0,37.419932,127.126290,470.0
1047,경기도,과천시,2022,3.0,37.429233,126.987721,486.0
1196,경기도,과천시,2023,3.0,37.429233,126.987721,479.0
...,...,...,...,...,...,...,...
953,충청남도,당진시,2021,20197.0,36.889675,126.645888,595.0
82,전라북도,김제시,2015,20843.0,35.803582,126.880646,587.0
1137,전라남도,해남군,2022,20944.0,34.573517,126.599270,459.0
988,전라남도,해남군,2021,21170.0,34.573517,126.599270,520.0


In [ ]:
print(farmmap_gdf.sample(1))

              UID          면적             법정동주소             팜맵ID  \
4181667  02808484  477.488255  경상남도 산청군 금서면 수철리  488603402400519   

                                                  geometry  
4181667  MULTIPOLYGON (((127.82361 35.40366, 127.82362 ...  


#### 합쳐진 데이터셋 생성

In [ ]:
## 합쳐진 데이터셋 생성

product_df_related_farmmap = product_df.copy()
product_df_related_farmmap['matched_farmmap'] = None

def get_related_farmmap(row):
    matched_farmmap = farmmap_collapsed[
        (farmmap_collapsed['sidonm'] == row['시도']) &
        (farmmap_collapsed['sggnm'] == row['시군구'])
    ]

    # row를 복사하고, matched farmmap을 새 컬럼으로 추가
    row_dict = row.to_dict()
    row_dict['matched_farmmap'] = matched_farmmap.to_dict(orient='records')  # 리스트로 저장

    return pd.Series(row_dict)
product_df_related_farmmap = product_df.apply(get_related_farmmap, axis=1)

> GEE 이용 NDVI 포인트별로 반환

In [ ]:
def get_ndvi(year, row):
    # 좌표를 포인트로 정의

    #polygons = []
    #print (row)
    if len(row) == 0:
      return None

    polygons = [x['geometry_x'] for x in row]
    # for farmmap in farmmaps:
    #   print (farmmap)
    #   gm = farmmap['geometry']
    #   polygons.append(gm)
    name = row[0]['name']

    ndvi_values = []
    for polygon in polygons:
      #mp = ee.Geometry.MultiPolygon(polygon)
      #mp = polygon#.to_crs(epsg=4326)
      # print (mp)
      #ee_polygons = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[polygon],crs='EPSG:5179'))
      center = polygon.centroid
      #center_wgs84 = gpd.GeoSeries([center], crs='EPSG:5179').to_crs('EPSG:4326').geometry[0]
      center_wgs84 = center
      ee_point = ee.Geometry.Point(center_wgs84.x, center_wgs84.y)
      ee_circle = ee_point.buffer(100000)  # 10,000 meters
      # print (ee_circle)

      mp = ee_circle#.geometry(ee_polygons)


      # 연도별 시작일과 종료일 정의
      start_date = f"{year}-04-01"  # 봄철 식생기
      end_date = f"{year}-10-31"    # 가을까지 포함

      ndvi_value = None

      try:
        # Sentinel-2 이미지 컬렉션 불러오기 (NDVI 계산 가능)


        collection = (
            ee.ImageCollection('NASA/HLS/HLSS30/v002') \
            .filterDate(start_date, end_date) \
            #.filterBounds(mp)
            .filterBounds(ee.Geometry.Rectangle([125.991211,33.063924, 129.440918,38.754083])) \
            .filter(ee.Filter.lt("CLOUD_COVERAGE", 20))
        )

        # 이미지 개수
        print (f"IMAGES : {collection.size().getInfo()}")

        # NDVI 계산: (NIR - RED) / (NIR + RED)
        def add_ndvi(img):
            ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
            return img.addBands(ndvi)

        ndvi_collection = collection.map(add_ndvi)

        # NDVI 밴드의 평균 계산
        mean_ndvi = ndvi_collection.select('NDVI').mean()

        # 포인트에서 NDVI 추출
        ndvi_value = mean_ndvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=mp,
            scale=10
        ).get('NDVI').getInfo()
      except Exception as E:
        print (E)
      finally:
        ndvi_value = None

      print (ndvi_value)
      print(f"NAME: {name} -> VAL: {ndvi_value}")
      ndvi_values.append(ndvi_value)

    return ndvi_values

##### 합쳐진 데이터셋에 NDVI 추가

In [ ]:
product_df_related_farmmap.head(2)

,시도,시군구,연도,면적_ha,위도,경도,10a당_생산량_kg,matched_farmmap
0,인천광역시,강화군,2015,8736.0,37.746568,126.488040,502.0,[]
1,광주광역시,광산구,2015,3012.0,35.139547,126.793633,508.0,"[{'adm_cd': '24050511', 'name': '광주광역시 광산구 송정1..."


In [ ]:
product_df_related_farmmap_ndvi = product_df_related_farmmap.copy()
product_df_related_farmmap_ndvi['ndvi'] = None

product_df_related_farmmap_ndvi = product_df_related_farmmap_ndvi[(product_df_related_farmmap_ndvi['연도'] > 2016)]# & (product_df_related_farmmap_ndvi['시도'] != '광주광역시')]
product_df_related_farmmap_ndvi['ndvi'] = product_df_related_farmmap.apply(lambda row: get_ndvi(row['연도'], row['matched_farmmap']), axis=1)

Streaming output truncated to the last 5000 lines.
NAME: 경기도 포천시 소흘읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 포천시 소흘읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 여주시 가남읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 -> VAL: None
IMAGES : 274
None
NAME: 경기도 연천군 연천읍 

In [ ]:
### 재정렬 / 컬럼명 정리

df = raw_df.rename(columns={
    'COMMODITY_DESC': 'crop',
    'YEAR': 'year',
    'STATE_NAME': 'state',
    "COUNTRY_NAME": "country",
    'COUNTY_NAME': 'county',
    'DOMAIN_DESC': 'domain',
    'STATISTICCAT_DESC': 'type',
    'UNIT_DESC': 'unit',
    'VALUE': 'value',
    'LOCATION_DESC': 'loc',
    'COUNTRY_CODE': 'country_code',
    'STATE_FIPS_CODE': 'state_fips',
    'COUNTY_CODE': 'county_code'
    }, inplace=False).copy()
df = df[['crop', 'year', 'country',  'country_code',  'state', 'state_fips',  'county', 'county_code', 'type', 'domain', 'unit', 'value', 'loc']].copy()
df['country_code'] = df['country_code'].astype(int)
df['state_fips'] = df['state_fips'].astype(int)
df['county_code'] = df['county_code'].astype(int)
df = df.sort_values(by=['year', 'state', 'county', 'crop', 'domain', 'type'], ascending=[True, True, True, True, True, True], inplace=False).copy().reset_index(drop=True)

df["type_unit"] = df["type"] + " [" + df["unit"] + "]"
df_pivot = df.pivot_table(index=["year", "state", "state_fips", "county", "county_code", "crop"],
    columns="type_unit",
    values="value"
).reset_index()

df_pivot.columns.name = None

#raw_data = df.copy()

#df.assign(C=df['type'])

df_pivot = df_pivot[["year", "state", "state_fips", "county", "county_code", "crop", "AREA HARVESTED [ACRES]", "AREA PLANTED [ACRES]", "YIELD [BU / ACRE]", "PRODUCTION [BU]"]]
df_pivot = df_pivot.rename(columns={
    'AREA HARVESTED [ACRES]': 'area_harvested',
    'AREA PLANTED [ACRES]': 'area_planted',
    'YIELD [BU / ACRE]': 'yield',
    'PRODUCTION [BU]': 'production'
}).copy()

df = df_pivot.reset_index(drop=True)

raw_data = df.copy()

## 유틸 함수

In [ ]:

### STATE_FIPS, COUNTY_FIPS 를 이용하여 LAT / LON, BOUNDARY 가져옴 ( CDMWF 사용 할 용도)
def find_boundary(state_fips, county_fips):
    filtered_gdf = gdf[(gdf['STATEFP'] == str(state_fips).zfill(2)) & (gdf['COUNTYFP'] == str(county_fips).zfill(3))]
    target_row = None
    if not filtered_gdf.empty:
        target_row = filtered_gdf.iloc[0]['geometry']
    else:
        return None

    if target_row is not None:
        (minx, miny, maxx, maxy) = target_row.bounds
        print(f"Bounds: {minx}, {miny}, {maxx}, {maxy}")
        return target_row

x = find_boundary(41, 1)
print(x.bounds)
# bound to lat lon
(minlat, minlon, maxlat, maxlon) = x.bounds
print(f"Bounds: {minlat}, {minlon}, {maxlat}, {maxlon}")

Bounds: -118.519477, 44.25509, -116.78371, 45.080746
(-118.519477, 44.25509, -116.78371, 45.080746)
Bounds: -118.519477, 44.25509, -116.78371, 45.080746


## Training

### 설정

In [ ]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [ ]:
## 데이터 정리 및 정규화, 데이터셋 생성

df = raw_data.copy()

## 데이터프레임중, 2000년 이후의 데이터만 남기기 ( NDVI )

df = df[df['year'] >= 2000].copy().reset_index(drop=True)


df['area_harvested'] = df['area_harvested'].replace(np.nan, 0)
df['yield'] = df['yield'].replace(np.nan, 0)
df['production'] = df['production'].replace(np.nan, 0)
df['area_planted'] = df['area_planted'].replace(np.nan, 0)

df['time_idx']  = df['year'] - df['year'].min()
df['group'] = df['crop'] + "_" + df['state'] + "_" + df['county']
df['group'] = df['group'].astype("category")
df['time_idx'] = df['time_idx'].astype("int64")

# drop na

df = df.dropna(subset=['area_harvested', 'area_planted', 'yield', 'production'], how='any').copy()
df = df.dropna(subset=['state', 'county', 'crop'], how='any').copy()
df = df.dropna(subset=['time_idx', 'group'], how='any').copy()
df = df.dropna(subset=['year'], how='any').copy()
df = df.dropna(subset=['state_fips'], how='any').copy()

df = df[['time_idx', 'group', 'year', 'state', 'county', 'crop', 'area_harvested', 'area_planted', 'yield', 'production']].copy()

sizes = df.groupby(['group']).size()

# drop groups with less than 14 rows
df = df[df['group'].isin(sizes[sizes >= 14].index)].copy()

<ipython-input-7-d880b861271e>:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sizes = df.groupby(['group']).size()


In [ ]:
df = df[(df['group'].isin(['CORN_FLORIDA_BROWARD']) == False)]

In [ ]:
max_prediction_length = 3
max_encoder_length = 6
training_cutoff = df["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],  # use all data up to the cutoff
    #data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="production",
    group_ids=["group"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["state", "crop", "county"],
    static_reals=[],
    # time_varying_known_reals=[col for col in df.columns if "ndvi_" in col or "prec_" in col or "temp_" in col],
    time_varying_known_reals=["area_harvested", "year"],
    time_varying_unknown_reals=["yield"],
    #variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    #time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    time_varying_unknown_categoricals=[],
    # time_varying_unknown_reals=[
    #     "volume",
    #     "log_volume",
    #     "industry_volume",
    #     "soda_volume",
    #     "avg_max_temp",
    #     "avg_volume_by_agency",
    #     "avg_volume_by_sku",
    # ],
    target_normalizer=GroupNormalizer(
        groups=["group"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
)

print (training)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=False)

# create dataloaders for model
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=1, persistent_workers=True)

val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=1, persistent_workers=True)

TimeSeriesDataSet[length=108632](
	time_idx='time_idx',
	target='production',
	group_ids=['group'],
	weight=None,
	max_encoder_length=6,
	min_encoder_length=3,
	min_prediction_idx=0,
	min_prediction_length=1,
	max_prediction_length=3,
	static_categoricals=['state', 'crop', 'county'],
	static_reals=[],
	time_varying_known_categoricals=None,
	time_varying_known_reals=['area_harvested', 'year'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['yield'],
	variable_groups=None,
	constant_fill_strategy=None,
	allow_missing_timesteps=True,
	lags=None,
	add_relative_time_idx=True,
	add_target_scales=True,
	add_encoder_length=True,
	target_normalizer=GroupNormalizer(
	method='standard',
	groups=['group'],
	center=True,
	scale_by_group=False,
	transformation='softplus',
	method_kwargs={}
),
	categorical_encoders={'__group_id__group': NaNLabelEncoder(add_nan=False, warn=True), 'group': NaNLabelEncoder(add_nan=False, warn=True), 'state': NaNLabelEncoder(add_nan=False, warn=True)

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1693: UserWarning: If predicting, no randomization should be possible - setting stop_randomization=True
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__group': 'CORN_IDAHO_OTHER (COMBINED) COUNTIES'}]
  warnings.warn(


In [ ]:
## OPTUNA를 이용해 최적화된 파라미터 로딩


# LOAD & USE
load_study = joblib.load(os.path.join(BASE_PATH, "optuna_best.pkl"))
print("Best trial until now:")
print(" Value: ", load_study.best_trial.value)
print(" Params: ")
for key, value in load_study.best_trial.params.items():
    print(f"    {key}: {value}")

Best trial until now:
 Value:  892647.9375
 Params: 
    gradient_clip_val: 0.9768071629453781
    hidden_size: 32
    dropout: 0.10371484042004216
    hidden_continuous_size: 11
    attention_head_size: 3
    learning_rate: 0.006890512969209501


### 학습

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor(logging_interval="step", log_momentum=True)  # log learning rate and momentum
logger = TensorBoardLogger(save_dir=os.path.join(BASE_PATH, 'tf_logger'))  # logging results to a tensorboard

pl.seed_everything(42)

## (*•؎ •*) 은하수를 여행하는 히치하이커 (*•؎ •*) ##

trainer = pl.Trainer(
    max_epochs=10,
    # max_steps=50,
    #accelerator="cpu", # tpu / cuda
    # accelerator="cuda",
    # accelerator="cpu",
    # accelerator=,
    enable_model_summary=False,
    gradient_clip_val=0.9768,
    #limit_train_batches=100,
    # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    log_every_n_steps=2,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.00689051296,
    hidden_size=32,
    attention_head_size=3,
    dropout=0.1037,
    hidden_continuous_size=11, # 연속 변수의 표현 차원수
    loss=QuantileLoss(),
    log_interval=8,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    log_val_interval=10,  # uncomment for best val on last log_val_interval evaluations
    # optimizer="Ranger",
    reduce_on_plateau_patience=4,
)


print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is rec

Number of parameters in network: 109.0k


### BEST Model 불러오기

In [ ]:
# BEST MODEL 불러오기

tft = TemporalFusionTransformer.load_from_checkpoint(os.path.join(BASE_PATH, "TFT/best_model.ckpt"))



/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


#### 실험

In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    #ckpt_path=os.path.join(BASE_PATH, "TFT/best_model.ckpt"),
    #fast_dev_run=True,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val_loss improved by 21791.688 >= min_delta = 0.0001. New best score: 489459.125
INFO:lightning.pytorch.callbacks.early_stopping:Metric val_loss improved by 21791.688 >= min_delta = 0.0001. New best score: 489459.125
INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### SAVE CHECKPOINT

In [ ]:
trainer.save_checkpoint(os.path.join(BASE_PATH, "TFT/best_model.ckpt"))

### 최적화

In [ ]:
import pickle

#from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# 파라미터 튜닝

study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path=os.path.join(BASE_PATH, "TFT"),
    n_trials=5, # 몇번 시도할지
    max_epochs=10,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,
    verbose=2,
    log_dir=os.path.join(BASE_PATH, "optuna"),
    # use Optuna to find ideal learning rate or use in-built learning rate finder
)

with open(os.path.join(BASE_PATH, "optuna_best.pkl"), "wb") as fout:
    pickle.dump(study, fout)


# show best hyperparameters
print(study.best_trial.params)

joblib.dump(study, os.path.join(BASE_PATH, "optuna_best.pkl"))


[I 2025-05-06 17:06:12,222] A new study created in memory with name: no-name-64bf09de-0609-4082-b80f-598d76dc7373
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/models/temporal_fusion_transformer/tuning.py:142: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gradient_clip_val = trial.suggest_loguniform(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/models/temporal_fusion_transformer/tuning.py:168: FutureWarning: suggest_uniform has

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-06 17:10:04,947] Trial 4 pruned. Trial was pruned at epoch 1.


{'gradient_clip_val': 0.9768071629453781, 'hidden_size': 32, 'dropout': 0.10371484042004216, 'hidden_continuous_size': 11, 'attention_head_size': 3, 'learning_rate': 0.006890512969209501}


### 훈련후 검증

In [ ]:
study

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
#best_model_path = trainer.checkpoint_callback.best_model_path
# best_model_path = "/content/drive/MyDrive/crop/tf_logger/lightning_logs/version_6/checkpoints/epoch=10-step=110.ckpt"
#print (best_model_path)


#best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
best_tft = TemporalFusionTransformer.load_from_checkpoint(os.path.join(BASE_PATH, "TFT/best_model.ckpt"))

# calcualte mean absolute error on validation set
predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))
MAE()(predictions.output, predictions.y)


/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU av

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 640 but got size 432 for tensor number 7 in the list.

In [ ]:
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True, return_index=True)

interpretation = best_tft.interpret_output(raw_predictions.output, reduction="sum")
best_tft.plot_interpretation(interpretation)


for idx in range(13):  # plot 10 examples
  plot_ind = best_tft.plot_prediction(raw_predictions.x, raw_predictions.output, idx=idx, add_loss_to_title=True, show_future_observed=True, plot_attention=False)
  idx_name = (raw_predictions.index['group'][idx])
  plot_ind.suptitle(f"Group: {idx_name}")
  # plot_ind.set(title=f"Group: {idx_name}")
  plot_ind.show()

In [ ]:
from pytorch_forecasting.metrics import SMAPE

# calcualte metric by which to display
#predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))

#print (predictions.output)

#y_pred = predictions.y[0].cpu()



# 예측
predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"), return_index=True)

# 예측값과 실제값 (둘 다 CPU로 옮겨서 계산)
y_pred = predictions.output.cpu()
y_true = predictions.y[0].cpu()

# SMAPE 인스턴스 생성
smape = SMAPE()

print (y_pred)
print (y_true)
# 전체 평균 SMAPE 계산
smape_score = smape(y_pred, y_true)

print(f"📊 SMAPE (전체 평균): {smape_score * 100:.2f}%")



# mean_losses = SMAPE(reduction="none")(predictions.output, predictions.y).mean(1)
# indices = mean_losses.argsort(descending=True)  # sort losses
# for idx in range(10):  # plot 10 examples
    # best_tft.plot_prediction(
    #     raw_predictions.x,
    #     raw_predictions.output,
    #     idx=indices[idx],
    #     add_loss_to_title=SMAPE(quantiles=best_tft.loss.quantiles),
    # )


predictions = best_tft.predict(val_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(predictions.x, predictions.output)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals)

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 640 but got size 432 for tensor number 7 in the list.